In [ ]:
# LTN: https://github.com/logictensornetworks/logictensornetworks/blob/master/examples/multiclass_classification/multiclass-singlelabel.ipynb
# Common.py : https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

In [1]:
!pip install PyTDC rdkit-pypi ltn keras==2.15.0 -qq
!wget https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import pandas as pd
import numpy as np
import ltn
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
tqdm.pandas()

# Utils Functions

In [5]:
# Threshold ref: https://pubs.acs.org/doi/epdf/10.1021/acs.jcim.3c01301
def label_th(pic50):
    classes = []
    for x in pic50:
        if x>=5:
            classes.append(1)
        else:
            classes.append(0)

    return np.asarray(classes)
class_map = {
    "blocks":1,
    "non-blocks":0,
}

### Data Acquisition

In [6]:
dataset_path = "/content/drive/MyDrive/Project/AI and Cardiology/Cardiotoxicity/Dataset"

In [7]:
!ls "{dataset_path}/External-Data"

CDK-external_test_set_pos.csv  eval_set_herg_70.csv	       herg_mmb_emb_h70.npz
CDK-herg60.csv		       external_test_set_pos.csv       Morgan-external_test_set_pos.csv
CDK-herg70.csv		       herg_mmb_emb_external_test.npz  Morgan-herg60.csv
eval_set_herg_60.csv	       herg_mmb_emb_h60.npz	       Morgan-herg70.csv


In [8]:
df = pd.read_csv(f"{dataset_path}/UniChemDB-Data/final-herg.csv")
df.dropna(subset = ['std_smiles'],inplace = True)
df.reset_index(drop = True,inplace = True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20409 entries, 0 to 20408
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          20388 non-null  object
 1   std_smiles  20409 non-null  object
 2   classes     20409 non-null  object
dtypes: object(3)
memory usage: 478.5+ KB


# Data



In [10]:
#External Test-1: https://github.com/Abdulk084/CardioTox/blob/master/data/external_test_set_pos.csv
test_pos_df = pd.read_csv(f"{dataset_path}/External-Data/external_test_set_pos.csv")

In [11]:
# External Test h70, h60 dataset: https://github.com/issararab/CToxPred/tree/main/data/raw/hERG
test_h60_df = pd.read_csv(f"{dataset_path}/External-Data/eval_set_herg_60.csv")
test_h70_df = pd.read_csv(f"{dataset_path}/External-Data/eval_set_herg_70.csv")


In [12]:
test_h60_df.head()

,InChl Key,SMILES,Source,pIC50
0,LIHJHFVXLZSRNK-UHFFFAOYSA-N,Cn1ccc(C[N+]2=CC(c3cccc(C(F)(F)F)c3)C=N2)n1,US Patent,5.647817
1,RXGDDWPITVSKDR-UHFFFAOYSA-N,CC(C)(C)OC(=O)N1CCN(c2nc3c([N+](=O)[O-])c(Br)c...,US Patent,5.602060
2,YRSBMPKJFDYYFO-UHFFFAOYSA-N,Fc1cccc(Oc2cc(C(F)(F)F)nc(N3CCc4nc[nH]c4C3)n2)c1,US Patent,5.599980
3,OMQQLDITRYIEHZ-UHFFFAOYSA-N,Cn1nccc1Cc1cn(-c2ccc(F)c(Cl)c2)nn1,US Patent,5.364516
4,BXBUTKPGTGJGTQ-YOEHRIQHSA-N,CNC[C@@H](c1ccc(Cl)c(Cl)c1)[C@@H](OC)c1cccc(NS...,US Patent,5.327902


In [13]:
#Threshold conversion
test_h60_df['target'] = label_th(test_h60_df.pIC50)
test_h70_df['target'] = label_th(test_h70_df.pIC50)

In [14]:
!ls "{dataset_path}"/UniChemDB-Data

CDK-unichemdb.csv  final-herg.csv  final-herg-split.csv  mmb_embeddings.npy  Morgan-unichemdb.csv


In [15]:
X = pd.read_csv(dataset_path+f"/UniChemDB-Data/CDK-unichemdb.csv").set_index("Name")
y = df['classes'].replace(class_map).values

In [16]:
X.shape, y.shape

((20409, 1024), (20409,))

In [17]:
!ls '{dataset_path}/External-Data'

CDK-external_test_set_pos.csv  eval_set_herg_70.csv	       herg_mmb_emb_h70.npz
CDK-herg60.csv		       external_test_set_pos.csv       Morgan-external_test_set_pos.csv
CDK-herg70.csv		       herg_mmb_emb_external_test.npz  Morgan-herg60.csv
eval_set_herg_60.csv	       herg_mmb_emb_h60.npz	       Morgan-herg70.csv


In [18]:
ext_pos_df = pd.read_csv(f'{dataset_path}/External-Data/CDK-external_test_set_pos.csv').rename(columns = {'Name':"target"})
ext_h60_df = pd.read_csv(f'{dataset_path}/External-Data/CDK-herg60.csv').rename(columns = {'Name':"target"})
ext_h70_df = pd.read_csv(f'{dataset_path}/External-Data/CDK-herg70.csv').rename(columns = {'Name':"target"})

In [19]:
batch_size = 64
ds_train = tf.data.Dataset.from_tensor_slices((X,y)).batch(batch_size)
idx = X.sample(1000).index
ds_test = tf.data.Dataset.from_tensor_slices((X.iloc[idx],y[idx])).batch(batch_size)

# LTN

Predicate with softmax `P(x,class)`

In [20]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16,16,8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs, training=False):
        x = inputs[0]
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)

logits_model = MLP(2)
p = ltn.Predicate.FromLogits(logits_model, activation_function="softmax", with_class_indexing=True)

Constants to index/iterate on the classes

In [21]:
class_A = ltn.Constant(0, trainable=False)
class_B = ltn.Constant(1, trainable=False)
# class_C = ltn.Constant(2, trainable=False)

Operators and axioms

In [22]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [23]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms(features, labels, training=False):
    x_A = ltn.Variable("x_A",features[labels==0])
    x_B = ltn.Variable("x_B",features[labels==1])
    # x_C = ltn.Variable("x_C",features[labels==2])
    axioms = [
        Forall(x_A,p([x_A,class_A],training=training)),
        Forall(x_B,p([x_B,class_B],training=training)),
        # Forall(x_C,p([x_C,class_C],training=training))
    ]
    for i in range(len(axioms)):
        if tf.math.is_nan(axioms[i].tensor):
            axioms[i].tensor  =0.0
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

Initialize all layers and the static graph

In [24]:
for features, labels in ds_test:
    print("Initial sat level %.5f"%axioms(features,labels))
    break

Initial sat level 0.49705


# Training

Define the metrics. While training, we measure:
1. The level of satisfiability of the Knowledge Base of the training data.
1. The level of satisfiability of the Knowledge Base of the test data.
3. The training accuracy.
4. The test accuracy.

In [25]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

Define the training and test step

In [26]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    sat = axioms(features, labels) # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['train_accuracy'](tf.one_hot(labels,2),predictions)

@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['test_accuracy'](tf.one_hot(labels,2),predictions)

Train

In [27]:
import commons

EPOCHS = 500

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="herg_cdk_results.csv",
    track_metrics=20
)

Epoch 0, train_sat_kb: 0.4808, test_sat_kb: 0.5472, train_accuracy: 0.5839, test_accuracy: 0.6780
Epoch 20, train_sat_kb: 0.6023, test_sat_kb: 0.5997, train_accuracy: 0.8037, test_accuracy: 0.7740
Epoch 40, train_sat_kb: 0.6534, test_sat_kb: 0.6426, train_accuracy: 0.8613, test_accuracy: 0.8280
Epoch 60, train_sat_kb: 0.6796, test_sat_kb: 0.6518, train_accuracy: 0.8872, test_accuracy: 0.8380
Epoch 80, train_sat_kb: 0.6978, test_sat_kb: 0.6929, train_accuracy: 0.9013, test_accuracy: 0.8750
Epoch 100, train_sat_kb: 0.7132, test_sat_kb: 0.7136, train_accuracy: 0.9122, test_accuracy: 0.8890
Epoch 120, train_sat_kb: 0.7233, test_sat_kb: 0.7197, train_accuracy: 0.9183, test_accuracy: 0.8940
Epoch 140, train_sat_kb: 0.7308, test_sat_kb: 0.7287, train_accuracy: 0.9248, test_accuracy: 0.8940
Epoch 160, train_sat_kb: 0.7419, test_sat_kb: 0.7357, train_accuracy: 0.9329, test_accuracy: 0.9080
Epoch 180, train_sat_kb: 0.7477, test_sat_kb: 0.7342, train_accuracy: 0.9334, test_accuracy: 0.9020
Epoch 

In [28]:
!ls "{dataset_path}/../Model-Weights"

hERG-Karim-CDK.keras  hERG-Karim-Morgan_CDK.keras  hERG-UniChemDB-Morgan.keras
hERG-Karim-MMB.keras  hERG-Karim-Morgan.keras


In [29]:
logits_model.save(f"{dataset_path}/../Model-Weights/hERG-UniChemDB-CDK.keras")

## Model Evaluation

In [30]:
from sklearn.metrics import (
    accuracy_score as ays,
    f1_score as fs,
    precision_score as ps,
    recall_score as rs,
    matthews_corrcoef as mcc,
    roc_auc_score as auc,
    balanced_accuracy_score,
    confusion_matrix.
)

In [35]:
def print_score(xtest,ytest,name):

    pred_test = logits_model.predict([xtest]).argmax(-1)

    auc_test = auc(ytest, pred_test)


    tn, fp, fn, tp = confusion_matrix(ytest, pred_test).ravel()

    specificity_test = tn / (tn + fp)

    sensitivity_test = tp / (tp + fn)

    NPV_test = tn / (tn + fn)

    PPV_test = tp / (tp + fp)
    Accuracy_test = ays(ytest, pred_test)
    Balanced_Accuracy_test = balanced_accuracy_score(ytest, pred_test)

    MCC_test= mcc(ytest, pred_test)

    CCR = (sensitivity_test+specificity_test)/2

    f1_score = fs(ytest, pred_test)
    print(f"MCC_test_{name}: " + str(MCC_test))
    print(f"NPV_test_{name}g: " + str(NPV_test))
    print(f"Accuracy_test_{name}: " + str(Accuracy_test))
    print(f"PPV_test_{name}: " + str(PPV_test))
    print(f"specificity_test_{name}: " + str(specificity_test))
    print(f"sensitivity_test_{name}: " + str(sensitivity_test))
    print(f"Balanced_Accuracy_test{name}: " + str(Balanced_Accuracy_test))

    print(f"Balanced_Accuracy_test{name}: " + str(Balanced_Accuracy_test))
    print(f"Correct Classification Rate (CCR){name}: " + str(CCR))
    print(f"F1 Score{name}: " + str(f1_score))



In [36]:
print_score(ext_pos_df.drop("target",axis = 1),ext_pos_df['target'],'External Data Test-1 (pos)')


2/2 [==============================] - 0s 9ms/step
MCC_test_External Data Test-1 (pos): 0.7217759492810224
NPV_test_External Data Test-1 (pos)g: 0.7222222222222222
Accuracy_test_External Data Test-1 (pos): 0.8636363636363636
PPV_test_External Data Test-1 (pos): 0.9615384615384616
specificity_test_External Data Test-1 (pos): 0.9285714285714286
sensitivity_test_External Data Test-1 (pos): 0.8333333333333334
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.8809523809523809
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.8809523809523809
Correct Classification Rate (CCR)External Data Test-1 (pos): 0.8809523809523809
F1 ScoreExternal Data Test-1 (pos): 0.8928571428571429


In [ ]:
print_score(ext_h60_df.drop("target",axis = 1),ext_h60_df['target'],'External hERG-60')

8/8 [==============================] - 0s 3ms/step
MCC_test_External hERG-60: 0.6304783133466723
NPV_test_External hERG-60g: 0.7202380952380952
Accuracy_test_External hERG-60: 0.796
PPV_test_External hERG-60: 0.9512195121951219
specificity_test_External hERG-60: 0.968
sensitivity_test_External hERG-60: 0.624
Balanced_Accuracy_testExternal hERG-60: 0.796
Balanced_Accuracy_testExternal hERG-60: 0.796
Correct Classification Rate (CCR)External hERG-60: 0.796
F1 ScoreExternal hERG-60: 0.753623188405797


In [ ]:
print_score(ext_h70_df.drop("target",axis = 1),ext_h70_df['target'],'External hERG-70')

15/15 [==============================] - 0s 2ms/step
MCC_test_External hERG-70: 0.6080807584874468
NPV_test_External hERG-70g: 0.7037037037037037
Accuracy_test_External hERG-70: 0.7906976744186046
PPV_test_External hERG-70: 0.9064039408866995
specificity_test_External hERG-70: 0.9090909090909091
sensitivity_test_External hERG-70: 0.696969696969697
Balanced_Accuracy_testExternal hERG-70: 0.803030303030303
Balanced_Accuracy_testExternal hERG-70: 0.803030303030303
Correct Classification Rate (CCR)External hERG-70: 0.803030303030303
F1 ScoreExternal hERG-70: 0.7880085653104926
